In [1]:
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torchvision import datasets, transforms
from collections import deque
from sklearn.metrics import root_mean_squared_error, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [43]:
#Ирисы
data_1 = pd.read_csv('DATA/iris.csv').sample(frac=1)
X = data_1.drop('species', axis=1)
y = data_1['species'].map({'setosa':0, 'versicolor':1, 'virginica':2})
X_tensor = torch.from_numpy(X.values)
y_tensor = torch.from_numpy(y.values).type(torch.int64)

In [51]:
class LinearLayer:
    """
    Слой линейных регрессий(нейронов) с выбором функции активации
    Параметры
    --------
    in_features: int
        Размерность предыдущего слоя. Если текущий слой первый, то размерность входного вектора

    out_features: int
        Размерность текущего слоя(размерность вектора на выходе слоя)
        
    activation_function: {'linear', 'ReLU', 'Tanh'}
        Вид функции активации

    start_weights: {'zeros', 'random', 'xavier_uniform'}
        Способ инициализации начальных весов и смещения нейронов

    bias: {True, False}, default=True
        Включает смещение в модель нейрона
    """
    def __init__(self,
                 in_features,
                 out_features,
                 activation_function,
                 start_weights,
                 bias=True):
        self.activation_function = activation_function
        self.weights_ = self._start_weights_type[start_weights](in_features, (in_features, out_features))
        self.bias_ = self._start_weights_type[start_weights](in_features, (out_features,)) if bias else None

    def __repr__(self):
        return (f'Linear(in_dim={self.weights_.shape[0]}, '
                f'out_dim={self.weights_.shape[1]}, '
                f'activation_F={self.activation_function})')
    
    def findNeuronsOutput(self, x):
        output = self.weights_.T @ x
        if self.bias_ is not None:
            output += self.bias_
        return output

    def findLayerOutput(self, neurons_out):
        return self._activation_function_type[self.activation_function](neurons_out)

    def findLayerDerivativeOutput(self, neurons_out):
        return self._activation_function_dir_type[self.activation_function](neurons_out)

    def layerForward(self, x):
        neurons_output = self.findNeuronsOutput(x)
        self.dir_output_ = self.findLayerDerivativeOutput(neurons_output)
        return self.findLayerOutput(neurons_output)

    def layerBackward(self, prev_layer):
        self.error_ = prev_layer.weights_ @ (prev_layer.error_ * prev_layer.dir_output_) 

    def layerStep(self, x, lr):
        error_dir = self.error_ * self.dir_output_
        self.weights_ -= lr * (x.reshape(-1,1) @ error_dir.reshape(1,-1))
        if self.bias_ is not None:
            self.bias_ -= lr * error_dir

    def setError(self, error):
        self.error_ = error
    
    @staticmethod
    def _zeroStartWeights(in_dim, shape):
        return torch.zeros(shape)

    @staticmethod
    def _randomStartWeights(in_dim, shape):
        return torch.randn(shape)

    @staticmethod
    def _xavierUniformStartWeights(in_dim, shape):
        k = np.sqrt(1/in_dim)
        return torch.rand(shape) * torch.sign(torch.rand(shape)-0.5) * k

    @staticmethod
    def _linActivationF(y):
        return y
    
    @staticmethod
    def _reLUActivationF(y):
        return torch.where(y>=0, y, 0)

    @staticmethod
    def _tanhActivationF(y):
        return torch.tanh(y)
    
    @staticmethod
    def _dirLinActivationF(y):
        return torch.ones(y.shape[0])

    @staticmethod
    def _dirReLUActivationF(y):
        return torch.where(y>=0, 1, 0)

    @staticmethod
    def _dirTanhActivationF(y):
        return -torch.pow(torch.tanh(y), 2) + 1

    _start_weights_type = {'zeros': _zeroStartWeights,
                           'random': _randomStartWeights,
                           'xavier_uniform': _xavierUniformStartWeights}
    _activation_function_type = {'linear': _linActivationF,
                                 'ReLU': _reLUActivationF,
                                 'Tanh': _tanhActivationF}
    _activation_function_dir_type = {'linear': _dirLinActivationF,
                                     'ReLU': _dirReLUActivationF,
                                     'Tanh': _dirTanhActivationF}

In [52]:
class LayerSequential:
    """
    Класс для реализации последовательности слоев нейронной сети с проходом вперед и назад.
    Возможен выбор Функции потерь: square(для регрессии), log(для классификации)
    """
    def __init__(self):
        self.layers_name = []
        self.layers = []

    def __repr__(self):
        return ''.join([f'({layer_name}): {layer}\n'
                        for layer_name, layer in zip(self.layers_name, self.layers)])

    def __getitem__(self, index):
        return self.layers[index]
    
    def add_module(self, layer_name, layer):
        self.layers_name.append(layer_name)
        self.layers.append(layer)

    def forward(self, x):
        self.layers_output = [x]
        for layer_name, layer in zip(self.layers_name, self.layers):
            x = layer.layerForward(x)
            self.layers_output.append(x)

    def backward(self, y, loss_function):
        model_error = self._loss_function_dir_type[loss_function](self.layers[-1].output_, y)
        self.layers[-1].setError(model_error)
        prev_layer = self.layers[-1]
        for layer in self.layers[-2::-1]:
            layer.layerBackward(prev_layer)
            prev_layer = layer

    def step(self, x, lr):
        for layer in self.layers:
            layer.layerStep(x, lr)
            x = layer.output_
        
    @staticmethod
    def _dirSquareError(x, y):
        return (x - y)

    @staticmethod
    def _dirLogError(x, y):
        return -y * torch.exp(-y*x) / (1 + torch.exp(-y*x))
    
    _loss_function_dir_type = {'square': _dirSquareError,
                               'log': _dirLogError}    

In [50]:
layers = torch.nn.Sequential()
layers.add_module('lin1', torch.nn.Linear(4, 6, dtype=torch.float64))
layers.add_module('relu1', torch.nn.ReLU())
layers.add_module('lin2', torch.nn.Linear(6, 6, dtype=torch.float64))
layers.add_module('relu2', torch.nn.ReLU())
layers.add_module('res', torch.nn.Linear(6, 3, dtype=torch.float64))
layers(X_tensor)

tensor([[ 0.0191, -0.2633,  0.2120],
        [ 0.0221, -0.2682,  0.2238],
        [ 0.0691, -0.3964,  0.2203],
        [ 0.0763, -0.3999,  0.2097],
        [ 0.0442, -0.3544,  0.1890],
        [ 0.0519, -0.3608,  0.2166],
        [ 0.0415, -0.3205,  0.2112],
        [ 0.0582, -0.3586,  0.2215],
        [ 0.0261, -0.2902,  0.2210],
        [ 0.0433, -0.3351,  0.2016],
        [ 0.0564, -0.3797,  0.2229],
        [ 0.0183, -0.2698,  0.1998],
        [ 0.0260, -0.2785,  0.2347],
        [ 0.0501, -0.3420,  0.2037],
        [ 0.0876, -0.4554,  0.1878],
        [ 0.0605, -0.3852,  0.2015],
        [ 0.0401, -0.3304,  0.1995],
        [ 0.0705, -0.4186,  0.2249],
        [ 0.0191, -0.2645,  0.2110],
        [ 0.0682, -0.3927,  0.2013],
        [ 0.0842, -0.4347,  0.2269],
        [ 0.0615, -0.3918,  0.1922],
        [ 0.0204, -0.2690,  0.2130],
        [ 0.0167, -0.2610,  0.2008],
        [ 0.0450, -0.3440,  0.2068],
        [ 0.0535, -0.3584,  0.2129],
        [ 0.0807, -0.4344,  0.1844],
 

In [54]:
uLayers = LayerSequential()
uLayers.add_module('lin1', LinearLayer(4,6,'ReLU','xavier_uniform'))
uLayers.add_module('lin2', LinearLayer(6,6,'ReLU','xavier_uniform'))
uLayers.add_module('res', LinearLayer(4,6,'linear','xavier_uniform'))
uLayers.forward(X_tensor)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x4 and 150x4)